### 4월 자치구 별 이동인구 분석
[데이터 소개]
* 1) 서울시 행정동별 이동 2024년 4월 데이터
* https://data.seoul.go.kr/dataVisual/seoul/seoulLivingMigration.do

* 2) 서울시 행정동 코드 데이터
* https://data.seoul.go.kr/dataVisual/seoul/seoulLivingMigration.do

[변수]
* 도착시간 : 0 -> "0:00~0:59"를 의미
* 출발 자치구 : 자치구 코드
* 성별 : F(여성), M(남성)
* 나이 : 10 ~ 79세 (5세 단위), 0 ~ 10세, 80세 이상으로 구분
* 유형 : H(야간상주지), W(주간상주지), E(기타)
* 평균 이동 시간 : 동일 열 내 이동인구의 월 단위 평균 이동 시간 (10분단위)
* 이동인구 : 추정 합산값으로 소수점 표출, 개인정보 비식별화를 위해 3명 미만의 경우 * 처리

#### 1. 데이터 불러오기
 - 출/퇴근시간 버스혼잡을 분산시키기 위한 노선 증설이므로 출/퇴근시간 데이터만 불러옴
 - 출근시간 (05:00 - 09:59), 퇴근시간 (17:00 - 21:59)으로 정의

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import scipy.stats as spst
from statsmodels.graphics.mosaicplot import mosaic

plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False

In [ ]:
#폴더 내 여러 파일 이름 리스트로 저장

path = "C:/Users/User/Desktop/생활이동_행정동_202404/"
file_list = os.listdir(path)
file_list

['생활이동_행정동_2024.04_00시.csv',
 '생활이동_행정동_2024.04_01시.csv',
 '생활이동_행정동_2024.04_02시.csv',
 '생활이동_행정동_2024.04_03시.csv',
 '생활이동_행정동_2024.04_04시.csv',
 '생활이동_행정동_2024.04_05시.csv',
 '생활이동_행정동_2024.04_06시.csv',
 '생활이동_행정동_2024.04_07시.csv',
 '생활이동_행정동_2024.04_08시.csv',
 '생활이동_행정동_2024.04_09시.csv',
 '생활이동_행정동_2024.04_10시.csv',
 '생활이동_행정동_2024.04_11시.csv',
 '생활이동_행정동_2024.04_12시.csv',
 '생활이동_행정동_2024.04_13시.csv',
 '생활이동_행정동_2024.04_14시.csv',
 '생활이동_행정동_2024.04_15시.csv',
 '생활이동_행정동_2024.04_16시.csv',
 '생활이동_행정동_2024.04_17시.csv',
 '생활이동_행정동_2024.04_18시.csv',
 '생활이동_행정동_2024.04_19시.csv',
 '생활이동_행정동_2024.04_20시.csv',
 '생활이동_행정동_2024.04_21시.csv',
 '생활이동_행정동_2024.04_22시.csv',
 '생활이동_행정동_2024.04_23시.csv']

In [ ]:
#출, 퇴근시간 데이터만 남기기 위해 리스트 요소 삭제
new_file_list = file_list.copy()

del new_file_list[0:5]
del new_file_list[-2:]
del new_file_list[5:12]

new_file_list

['생활이동_행정동_2024.04_05시.csv',
 '생활이동_행정동_2024.04_06시.csv',
 '생활이동_행정동_2024.04_07시.csv',
 '생활이동_행정동_2024.04_08시.csv',
 '생활이동_행정동_2024.04_09시.csv',
 '생활이동_행정동_2024.04_17시.csv',
 '생활이동_행정동_2024.04_18시.csv',
 '생활이동_행정동_2024.04_19시.csv',
 '생활이동_행정동_2024.04_20시.csv',
 '생활이동_행정동_2024.04_21시.csv']

In [ ]:
dong_moving = pd.DataFrame()

for i in new_file_list:
  data = pd.read_csv(path + i, encoding = 'cp949')
  dong_moving = pd.concat([dong_moving, data])

dong_moving = dong_moving.reset_index(drop = True)

In [ ]:
#행정동 코드 데이터
dong_info = pd.read_excel("C:/Users/User/Desktop/서울생활이동데이터_행정동코드_20210907.xlsx")

#### 2. 데이터 살펴보기

In [ ]:
dong_moving.head()

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202404,일,5,1101053,1101053,F,20,EE,13,*
1,202404,일,5,1101053,1101053,F,20,WE,4,*
2,202404,일,5,1101053,1101053,F,25,WW,1,*
3,202404,일,5,1101053,1101053,F,40,HW,30,3.41
4,202404,일,5,1101053,1101053,F,50,HH,42,3.10


In [ ]:
dong_moving.tail()

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
46958001,202404,목,21,39000,1124065,F,65,EH,168,4.63
46958002,202404,목,21,39000,1124065,M,20,EH,198,*
46958003,202404,목,21,39000,1124065,M,20,HE,233,*
46958004,202404,목,21,39000,1124065,M,30,EE,243,*
46958005,202404,목,21,39000,1124065,M,40,EH,224,*


In [ ]:
dong_info.head()

,시도,시군구,읍면동,name,full_name
0,11000,11010,1101053,사직동,서울특별시 종로구 사직동
1,11000,11010,1101054,삼청동,서울특별시 종로구 삼청동
2,11000,11010,1101055,부암동,서울특별시 종로구 부암동
3,11000,11010,1101056,평창동,서울특별시 종로구 평창동
4,11000,11010,1101057,무악동,서울특별시 종로구 무악동


In [ ]:
dong_info.tail()

,시도,시군구,읍면동,name,full_name
1147,35000,35000,35000,전라북도,전라북도
1148,36000,36000,36000,전라남도,전라남도
1149,37000,37000,37000,경상북도,경상북도
1150,38000,38000,38000,경상남도,경상남도
1151,39000,39000,39000,제주특별자치도,제주특별자치도


In [ ]:
print(dong_moving.shape)
print('*' * 50)
dong_moving.info()

(46958006, 10)
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46958006 entries, 0 to 46958005
Data columns (total 10 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   대상연월         int64 
 1   요일           object
 2   도착시간         int64 
 3   출발 행정동 코드    int64 
 4   도착 행정동 코드    int64 
 5   성별           object
 6   나이           int64 
 7   이동유형         object
 8   평균 이동 시간(분)  int64 
 9   이동인구(합)      object
dtypes: int64(6), object(4)
memory usage: 3.5+ GB


In [ ]:
print(dong_info.shape)
print('*' * 50)
dong_info.info()

(1152, 5)
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   시도         1152 non-null   int64 
 1   시군구        1152 non-null   int64 
 2   읍면동        1152 non-null   int64 
 3   name       1152 non-null   object
 4   full_name  1152 non-null   object
dtypes: int64(3), object(2)
memory usage: 45.1+ KB


In [ ]:
#결측치 확인
dong_moving.isna().sum()

대상연월           0
요일             0
도착시간           0
출발 행정동 코드      0
도착 행정동 코드      0
성별             0
나이             0
이동유형           0
평균 이동 시간(분)    0
이동인구(합)        0
dtype: int64

In [ ]:
dong_info.isna().sum()

시도           0
시군구          0
읍면동          0
name         0
full_name    0
dtype: int64

In [ ]:
#기초통계량 확인
dong_moving.describe().T

,count,mean,std,min,25%,50%,75%,max
대상연월,46958006.0,2.024040e+05,0.000000,202404.0,202404.0,202404.0,202404.0,202404.0
도착시간,46958006.0,1.519232e+01,5.352281,5.0,9.0,18.0,19.0,21.0
출발 행정동 코드,46958006.0,1.410300e+06,731865.382675,21000.0,1108084.0,1117071.0,1124055.0,3138041.0
도착 행정동 코드,46958006.0,1.422716e+06,737508.494720,21000.0,1109065.0,1117072.0,1124059.0,3138041.0
나이,46958006.0,4.204433e+01,16.620401,0.0,30.0,40.0,55.0,80.0
평균 이동 시간(분),46958006.0,6.623992e+01,62.260042,1.0,29.0,52.0,84.0,2979.0


 - *은 3명이하 -> 이동인구(합)의 최빈값 or 평균을 알 수 없음 (type이 string이라서) -> 중위값 1.5로 대체

In [ ]:
dong_moving = dong_moving.replace({'이동인구(합)':'*'}, 1.5)
dong_moving['이동인구(합)'] = dong_moving['이동인구(합)'].astype(float)
dong_moving['이동인구(합)'].value_counts()

이동인구(합)
1.50      10955530
6.05       1042639
3.30        991968
3.29        831636
3.28        652211
            ...   
583.63           1
428.11           1
429.57           1
530.34           1
209.57           1
Name: count, Length: 29136, dtype: int64

In [ ]:
#범주형 컬럼 값 확인하기
#요일 컬럼에 금요일, 토요일 없음
col_list = list(dong_moving.select_dtypes(include = 'object').columns)
for i in col_list:
    display(dong_moving[i].value_counts())

요일
화    11017966
월    10518744
목     9448754
수     9144086
일     6828456
Name: count, dtype: int64

성별
M    25320729
F    21637277
Name: count, dtype: int64

이동유형
EH    9678394
WH    8011587
HW    7606416
HE    7575428
EE    6054179
WE    3444754
EW    2057340
WW    1363679
HH    1166229
Name: count, dtype: int64

#### 서울지역의 데이터만 추출

 - Raw Data는 서울시에서의 이동 뿐 아니라 서울에서 나가거나 들어오는것도 포함
 - 서울시 데이터만 필요하므로 출발 행정동 코드 와 도착 행정동 코드가 서울지역인것만 추출
 - 행정동 코드 = 시군구코드 + 숫자 2자리
 - 시군구 코드 -> (서울:11, 부산:26, 대구:27, 인천:28, 광주:29, 대전:30, 울산:31)

In [ ]:
dong_moving.head()

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합)
0,202404,일,5,1101053,1101053,F,20,EE,13,1.50
1,202404,일,5,1101053,1101053,F,20,WE,4,1.50
2,202404,일,5,1101053,1101053,F,25,WW,1,1.50
3,202404,일,5,1101053,1101053,F,40,HW,30,3.41
4,202404,일,5,1101053,1101053,F,50,HH,42,3.10


In [ ]:
dong_location = dong_moving.loc[(dong_moving['출발 행정동 코드'] < 2000000) & (dong_moving['도착 행정동 코드'] < 2000000)]
dong_location['출발 행정동 코드'].value_counts()
dong_location['도착 행정동 코드'].value_counts()

도착 행정동 코드
1123064    294770
1119054    289298
1101061    257102
1114066    224472
1122053    220929
            ...  
39000       16395
1101069     13339
1122056     11670
26000        9595
1125070      8191
Name: count, Length: 438, dtype: int64

In [ ]:
# dong_location = dong_location[(dong_location['출발 행정동 코드']) & (dong_location['도착행정동코드'])]

In [ ]:
dong_info = dong_info.loc[dong_info['읍면동'] < 2000000]
dong_info['읍면동'].value_counts()

읍면동
1101053    1
1119061    1
1119073    1
1119072    1
1119071    1
          ..
1109075    1
1109074    1
1109073    1
1109072    1
39000      1
Name: count, Length: 438, dtype: int64

In [ ]:
# 2000000 이하로 데이터 필터링 했는데 39000같이 필터링 안된 데이터 존재(서울 읍면동 코드는 1000000단위)
dong_info = dong_info[dong_info['읍면동'] > 1000000]
dong_info['읍면동'].value_counts()

읍면동
1101053    1
1119065    1
1119062    1
1119061    1
1119056    1
          ..
1109071    1
1109070    1
1109069    1
1109065    1
1125074    1
Name: count, Length: 424, dtype: int64

#### 데이터 병합을 위한 이름변경

In [ ]:
dong_info = dong_info.rename(columns = {'읍면동':'출발 행정동 코드'})
dong_info

,시도,시군구,출발 행정동 코드,name,full_name
0,11000,11010,1101053,사직동,서울특별시 종로구 사직동
1,11000,11010,1101054,삼청동,서울특별시 종로구 삼청동
2,11000,11010,1101055,부암동,서울특별시 종로구 부암동
3,11000,11010,1101056,평창동,서울특별시 종로구 평창동
4,11000,11010,1101057,무악동,서울특별시 종로구 무악동
...,...,...,...,...,...
419,11000,11250,1125070,둔촌1동,서울특별시 강동구 둔촌1동
420,11000,11250,1125071,둔촌2동,서울특별시 강동구 둔촌2동
421,11000,11250,1125072,암사1동,서울특별시 강동구 암사1동
422,11000,11250,1125073,천호2동,서울특별시 강동구 천호2동


In [ ]:
dong_df = pd.merge(dong_location, dong_info, on = '출발 행정동 코드', how = 'left')
dong_df

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합),시도,시군구,name,full_name
0,202404,일,5,1101053,1101053,F,20,EE,13,1.50,11000.0,11010.0,사직동,서울특별시 종로구 사직동
1,202404,일,5,1101053,1101053,F,20,WE,4,1.50,11000.0,11010.0,사직동,서울특별시 종로구 사직동
2,202404,일,5,1101053,1101053,F,25,WW,1,1.50,11000.0,11010.0,사직동,서울특별시 종로구 사직동
3,202404,일,5,1101053,1101053,F,40,HW,30,3.41,11000.0,11010.0,사직동,서울특별시 종로구 사직동
4,202404,일,5,1101053,1101053,F,50,HH,42,3.10,11000.0,11010.0,사직동,서울특별시 종로구 사직동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31296217,202404,목,21,39000,1124065,F,65,EH,168,4.63,NaN,NaN,NaN,NaN
31296218,202404,목,21,39000,1124065,M,20,EH,198,1.50,NaN,NaN,NaN,NaN
31296219,202404,목,21,39000,1124065,M,20,HE,233,1.50,NaN,NaN,NaN,NaN
31296220,202404,목,21,39000,1124065,M,30,EE,243,1.50,NaN,NaN,NaN,NaN


In [ ]:
## left join 사용하여 dong_location에 결측치 발생시킴
## dong_info로 서울시 데이터로만 필터링 하기위함
dong_df = dong_df.dropna(axis = 0)
dong_df

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합),시도,시군구,name,full_name
0,202404,일,5,1101053,1101053,F,20,EE,13,1.50,11000.0,11010.0,사직동,서울특별시 종로구 사직동
1,202404,일,5,1101053,1101053,F,20,WE,4,1.50,11000.0,11010.0,사직동,서울특별시 종로구 사직동
2,202404,일,5,1101053,1101053,F,25,WW,1,1.50,11000.0,11010.0,사직동,서울특별시 종로구 사직동
3,202404,일,5,1101053,1101053,F,40,HW,30,3.41,11000.0,11010.0,사직동,서울특별시 종로구 사직동
4,202404,일,5,1101053,1101053,F,50,HH,42,3.10,11000.0,11010.0,사직동,서울특별시 종로구 사직동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31279628,202404,목,21,1125074,1106087,F,20,WH,79,4.42,11000.0,11250.0,길동,서울특별시 강동구 길동
31279629,202404,목,21,1125074,1106087,F,35,WH,36,3.05,11000.0,11250.0,길동,서울특별시 강동구 길동
31279630,202404,목,21,1125074,1106087,F,50,EE,43,9.45,11000.0,11250.0,길동,서울특별시 강동구 길동
31279631,202404,목,21,1125074,1106087,F,50,WH,74,3.14,11000.0,11250.0,길동,서울특별시 강동구 길동


In [ ]:
#도착 행정동 코드 서울 외부인 데이터 아직 필터링 안되었다.
dong_df = dong_df.loc[dong_df['도착 행정동 코드'] > 1000000]

In [ ]:
# 시도, 시군구 컬럼은 필요없다고 판단되어 삭제
drop_cols = ['시도', '시군구']
dong_df = dong_df.drop(drop_cols, axis = 1)
dong_df

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합),name,full_name
0,202404,일,5,1101053,1101053,F,20,EE,13,1.50,사직동,서울특별시 종로구 사직동
1,202404,일,5,1101053,1101053,F,20,WE,4,1.50,사직동,서울특별시 종로구 사직동
2,202404,일,5,1101053,1101053,F,25,WW,1,1.50,사직동,서울특별시 종로구 사직동
3,202404,일,5,1101053,1101053,F,40,HW,30,3.41,사직동,서울특별시 종로구 사직동
4,202404,일,5,1101053,1101053,F,50,HH,42,3.10,사직동,서울특별시 종로구 사직동
...,...,...,...,...,...,...,...,...,...,...,...,...
31279628,202404,목,21,1125074,1106087,F,20,WH,79,4.42,길동,서울특별시 강동구 길동
31279629,202404,목,21,1125074,1106087,F,35,WH,36,3.05,길동,서울특별시 강동구 길동
31279630,202404,목,21,1125074,1106087,F,50,EE,43,9.45,길동,서울특별시 강동구 길동
31279631,202404,목,21,1125074,1106087,F,50,WH,74,3.14,길동,서울특별시 강동구 길동


In [ ]:
#해당 데이터프레임 csv파일로 저장
dong_df.to_csv("dong_df.csv", index = False)

In [ ]:
# 피클파일로도 저장
import joblib

joblib.dump(dong_df, 'dong_df.pkl')

['dong_df.pkl']

In [ ]:
joblib.load("C:/Users/User/Desktop/dong_df.pkl")

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합),name,full_name
0,202404,일,5,1101053,1101053,F,20,EE,13,1.50,사직동,서울특별시 종로구 사직동
1,202404,일,5,1101053,1101053,F,20,WE,4,1.50,사직동,서울특별시 종로구 사직동
2,202404,일,5,1101053,1101053,F,25,WW,1,1.50,사직동,서울특별시 종로구 사직동
3,202404,일,5,1101053,1101053,F,40,HW,30,3.41,사직동,서울특별시 종로구 사직동
4,202404,일,5,1101053,1101053,F,50,HH,42,3.10,사직동,서울특별시 종로구 사직동
...,...,...,...,...,...,...,...,...,...,...,...,...
31279628,202404,목,21,1125074,1106087,F,20,WH,79,4.42,길동,서울특별시 강동구 길동
31279629,202404,목,21,1125074,1106087,F,35,WH,36,3.05,길동,서울특별시 강동구 길동
31279630,202404,목,21,1125074,1106087,F,50,EE,43,9.45,길동,서울특별시 강동구 길동
31279631,202404,목,21,1125074,1106087,F,50,WH,74,3.14,길동,서울특별시 강동구 길동
